# Data cleaning 

In [1]:
import pandas as pd
import os

import numpy as np

In [2]:
from use_cases.contributions import create_table_contributions, to_sql as con_to_sql
# 
from use_cases.emotions import create_table_emotions, to_sql as emo_to_sql 
from use_cases.personal_needs import create_table_personal_needs, to_sql as per_need_to_sql
from use_cases.dialogues import create_table_dialogues
from use_cases.country_needs import create_table_country_needs, to_sql as count_need_to_sql
from use_cases.persons import create_table_persons
from use_cases.individuals import create_table_individuals
from use_cases.pairs import create_pair_token
from use_cases.persons_dialogues import create_table_persons_dialogues

import use_cases.utils.textools as tt


pd.options.mode.chained_assignment = None 

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading Data Surveys

In [3]:
# chunksize   = 100 # Just for debugging
# survey_path = './data/BBDD_Dialogos.csv' #  chunksize=chunksize,
# survey = pd.read_csv(survey_path, low_memory=False)
# for survey in survey: break

survey_path = './data/BBDD_Dialogos.xlsx'
survey = pd.read_excel(survey_path, 'Hoja1')

filter_dialogue = pd.read_excel(survey_path, 'Filtro Diálogos')
filter_person = pd.read_excel(survey_path, 'Filtro Personas')

ind_online_survey_path = './data/Consulta_Individual_online_v2.xlsx'
ind_online_survey = pd.read_excel(ind_online_survey_path, 'Sheet1')
    
ind_survey_path = './data/Base_final_digitación_Consultas.xlsx'
ind_survey = pd.read_excel(ind_survey_path, 'CONSULTAS')

filter_individual_path = './data/Filtro_individuales.xlsx'
filter_digitilized_individual = pd.read_excel(filter_individual_path, 'Hoja1')

## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Persons Table (Dialogue)

In [223]:
%%time
persons_table = create_table_persons(survey)

CPU times: user 3min 28s, sys: 809 ms, total: 3min 28s
Wall time: 3min 29s


In [233]:
persons_table.sample()

,id,age,sex,level,comuna_id,age_range
6459,vp/tbtxqcfvf,51,h,Técnica completa,4301,50 a 59


Ahora chequearemos las tablas, este proceso se aplica a todas las tablas que se están procesando. Lo que hace es revisar si es que hay alguna id nula o si hay algún dato nulo (nr, NR, NaN, nan y np.nan en alguna parte de los datos (esto produce ruido al procesar, el vacio se definió como texto vacio.

La función retorna como dataframe con las filas con problemas, si retorna dataframe vacio entonces está correcto.

In [225]:
tt.last_sanity_check(persons_table, 'id')

(Empty DataFrame
 Columns: [id, diag_id, age, sex, level, comuna_id, age_range]
 Index: [],
 [],
 Empty DataFrame
 Columns: [id, diag_id, age, sex, level, comuna_id, age_range]
 Index: [])

### Dialogues

In [226]:
%%time
dialogues_table = create_table_dialogues(survey, filter_dialogue)

CPU times: user 2.83 s, sys: 16.2 ms, total: 2.84 s
Wall time: 2.85 s


In [227]:
dialogues_table.sample(2)

,id,date,init_time,end_time,location,address,comuna_id,n_members,group_name,valid
8539,enc_u_2823536,2020-01-14 00:00:00,17:35:00,19:05:00,caleta punta arenas,sede caleta pta arenas s/n,2301,5,usuarios de oficina movil de bienes nacionales,True
5173,enc_u_4563064441809607788,2020-01-25 00:00:00,20:00:00,22:00:00,casa particular,latorre 824,15101,4,usuarios de oficina movil de bienes nacionales,True


In [228]:
tt.last_sanity_check(dialogues_table, 'id')

(Empty DataFrame
 Columns: [id, date, init_time, end_time, location, address, comuna_id, n_members, group_name, valid]
 Index: [],
 [],
 Empty DataFrame
 Columns: [id, date, init_time, end_time, location, address, comuna_id, n_members, group_name, valid]
 Index: [])

### Persons Dialogues

In [229]:
%%time
persons_table, persons_dialogues_table = create_table_persons_dialogues(persons_table)

CPU times: user 82.7 ms, sys: 0 ns, total: 82.7 ms
Wall time: 81 ms


In [230]:
persons_dialogues_table.sample(2)

,person_id,diag_id
1411,vjls10r/2vhy,enc_u_4555076830717464193
3771,es11fov4jckv,enc_u_4549112060755676119


In [231]:
tt.last_sanity_check(persons_dialogues_table, 'person_id', ['person_id','diag_id'])

(Empty DataFrame
 Columns: [person_id, diag_id]
 Index: [],
 Empty DataFrame
 Columns: [person_id, diag_id]
 Index: [],
 Empty DataFrame
 Columns: [person_id, diag_id]
 Index: [])

### Individuals

In [4]:
%%time
individuals_table = create_table_individuals(ind_online_survey, ind_survey, filter_digitilized_individual)

CPU times: user 12.9 s, sys: 44.4 ms, total: 13 s
Wall time: 13 s


In [6]:
individuals_table.sample(2)

,id,date,age,comuna_id,level,is_valid,age_range,online
2095,3089227,17-01-2020,95,4201,MedInc,True,70 o más,False
11966,5838800,06-03-2020,68,6111,MedComp/TecInc,True,60 a 69,False


In [10]:
tt.last_sanity_check(individuals_table,'id')

(Empty DataFrame
 Columns: [id, date, age, comuna_id, level, is_valid, age_range, online]
 Index: [],
 [],
 Empty DataFrame
 Columns: [id, date, age, comuna_id, level, is_valid, age_range, online]
 Index: [])

### Emotions

In [241]:
%%time
emotions_table = create_table_emotions(survey, ind_survey_path, ind_online_survey)

CPU times: user 2min 59s, sys: 16 s, total: 3min 15s
Wall time: 3min 16s


In [242]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
3543,3519,enc_u_4544993731029250973,,inseguridad,[inseguridad],inseguridad,se observa en el control del orden publico,"[observa, control, orden, publico]",True
7500,32761,enc_u_4553455014962674487,,trabajo,[trabajo],trabajo,mas trabajo y mayor mano de obra chilena,"[mas, trabajo, mayor, mano, obra, chilena]",True


In [243]:
tt.last_sanity_check(emotions_table, 'id',['diag_id', 'ind_id'], ['name_tokens', 'exp_tokens'])

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(Empty DataFrame
 Columns: [id, diag_id, ind_id, name, name_tokens, macro, exp, exp_tokens, is_online]
 Index: [],
 Empty DataFrame
 Columns: [id, diag_id, ind_id, name, name_tokens, macro, exp, exp_tokens, is_online]
 Index: [],
 Empty DataFrame
 Columns: [id, diag_id, ind_id, name, macro, exp, is_online]
 Index: [])

### Emotion Pair

In [244]:
%%time
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

CPU times: user 14.5 s, sys: 92.2 ms, total: 14.6 s
Wall time: 14.6 s


In [245]:
emotion_pair.sample(3)

,id,emotion_id,word_1,word_2
37017,37018,6733,cuenta,pasando
91645,91646,16324,vandalismo,muchas
14418,14419,2547,terminar,estallido


### Country Needs

In [247]:
%%time
country_needs = create_table_country_needs(survey, ind_survey, ind_survey_path, ind_online_survey_path)

CPU times: user 5min 53s, sys: 27.6 s, total: 6min 21s
Wall time: 6min 21s


In [248]:
country_needs.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
4857,99494,,2728808,educacion,[educacion],,,,hay que deshacerse de la erronea vision que el...,"[deshacerse, erronea, vision, enfasis, educaci...",toda la comunidad,1,False
7822,27508,enc_u_4556938160807774586,,cambios de la clase politica,"[cambios, clase, politica]",,elecciones anticipadas de presidente y todo el...,"[elecciones, anticipadas, presidente, congreso...",,,,5,False


In [249]:
tt.last_sanity_check(country_needs, 'id',['diag_id', 'ind_id'], ['name_tokens', 'exp_tokens', 'role_tokens'])

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(Empty DataFrame
 Columns: [id, diag_id, ind_id, name, name_tokens, macro, exp, exp_tokens, role, role_tokens, actor, priority, is_online]
 Index: [],
 Empty DataFrame
 Columns: [id, diag_id, ind_id, name, name_tokens, macro, exp, exp_tokens, role, role_tokens, actor, priority, is_online]
 Index: [],
 Empty DataFrame
 Columns: [id, diag_id, ind_id, name, macro, exp, role, actor, priority, is_online]
 Index: [])

### Country Need Role Pair

In [250]:
%%time
country_need_role_pair = create_pair_token(country_needs, 'role_tokens', 'country_need_id')

CPU times: user 18.8 s, sys: 8.27 ms, total: 18.8 s
Wall time: 18.9 s


In [251]:
country_need_role_pair.sample(2)

,id,country_need_id,word_1,word_2
110197,110198,35532,debiles,mas
191268,191269,63352,elaborar,proceso


### Country Need Explanation Pair

In [252]:
%%time
country_need_exp_pair = create_pair_token(country_needs, 'exp_tokens', 'country_need_id')

CPU times: user 20 s, sys: 48.4 ms, total: 20 s
Wall time: 20.1 s


In [253]:
country_need_exp_pair.sample(2)

,id,country_need_id,word_1,word_2
201823,201824,44474,sistema,prevision
20464,20465,3018,garantizar,atencion


### Family/Personal Needs

In [254]:
%%time
personal_needs = create_table_personal_needs(survey, ind_survey, ind_online_survey)

CPU times: user 3min 16s, sys: 18.4 s, total: 3min 35s
Wall time: 3min 36s


In [258]:
personal_needs.sample(3)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
4388,16146,enc_u_4556195194809673278,,salud,[salud],hospital,[hospital],salud,2,False
5687,17309,enc_u_4587251749157123009,,endeudamiento para estudiar (cae),"[endeudamiento, estudiar, cae]",la mayoria de los nietos que estan en la univ...,"[mayoria, nietos, estan, universidad, endeudan...",endeudamiento para estudiar (cae),1,False
4812,28085,enc_u_4561146068508829205,,salud\nagua,"[salud, agua]",mejor salud para la gente de campo y 3 era eda...,"[mejor, salud, gente, campo, edad, poder, rega...",salud\nagua,0,False


In [256]:
tt.last_sanity_check(personal_needs, 'id',['diag_id', 'ind_id'], ['name_tokens', 'exp_tokens'])

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(Empty DataFrame
 Columns: [id, diag_id, ind_id, name, name_tokens, exp, exp_tokens, macro, priority, is_online]
 Index: [],
 Empty DataFrame
 Columns: [id, diag_id, ind_id, name, name_tokens, exp, exp_tokens, macro, priority, is_online]
 Index: [],
 Empty DataFrame
 Columns: [id, diag_id, ind_id, name, exp, macro, priority, is_online]
 Index: [])

### Family/Personal Need Pair

In [257]:
%%time
personal_need_pair = create_pair_token(personal_needs, 'exp_tokens', 'personal_need_id')

CPU times: user 17.1 s, sys: 88.7 ms, total: 17.2 s
Wall time: 17.2 s


In [259]:
personal_need_pair.sample(3)

,id,personal_need_id,word_1,word_2
347423,347424,59582,calidad,vida
282299,282300,46653,civica,sexual
160740,160741,25792,pocas,horas


### Contributions

In [260]:
%%time
contributions = create_table_contributions(survey, ind_survey, ind_online_survey)

CPU times: user 1min 20s, sys: 7.37 s, total: 1min 27s
Wall time: 1min 28s


In [261]:
contributions.sample(3)

,id,diag_id,ind_id,text,tokens,macro,is_online
8483,75103,,4195087,dando a conocer mi opinion,"[dando, conocer, opinion]",dando a conocer mi opinion,False
10571,10571,enc_u_4559690460153952011,,trabajando dia a dia,"[trabajando, dia, dia]",trabajando dia a dia,False
8035,61331,enc_u_4561267516712062710,,cuidar el medio ambiente y saber vivir en demo...,"[cuidar, medio, ambiente, saber, vivir, democr...",cuidar el medio ambiente y saber vivir en demo...,False


In [262]:
tt.last_sanity_check(contributions, 'id',['diag_id', 'ind_id'], ['tokens'])

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(Empty DataFrame
 Columns: [id, diag_id, ind_id, text, tokens, macro, is_online]
 Index: [],
 Empty DataFrame
 Columns: [id, diag_id, ind_id, text, tokens, macro, is_online]
 Index: [],
 Empty DataFrame
 Columns: [id, diag_id, ind_id, text, macro, is_online]
 Index: [])

## Saving `.csv`

In [8]:
out_dir = './out'
os.makedirs(out_dir, exist_ok=True)

In [264]:
contributions.to_csv(os.path.join(out_dir, 'contributions.csv'), index=False)
personal_need_pair.to_csv(os.path.join(out_dir, 'personal_needs_pairs.csv'), index=False)
personal_needs.to_csv(os.path.join(out_dir, 'personal_needs.csv'), index=False)
country_need_exp_pair.to_csv(os.path.join(out_dir, 'country_needs_exp_pairs.csv'), index=False)
country_need_role_pair.to_csv(os.path.join(out_dir, 'country_needs_role_pairs.csv'), index=False)
country_needs.to_csv(os.path.join(out_dir, 'country_needs.csv'), index=False)
emotion_pair.to_csv(os.path.join(out_dir, 'emotion_pair.csv'), index=False)
emotions_table.to_csv(os.path.join(out_dir, 'emotions.csv'), index=False)
individuals_table.to_csv(os.path.join(out_dir, 'individuals.csv'), index=False)
dialogues_table.to_csv(os.path.join(out_dir, 'dialogues.csv'), index=False)
persons_table.to_csv(os.path.join(out_dir, 'persons.csv'), index=False)
persons_dialogues_table.to_csv(os.path.join(out_dir, 'persons_dialogues.csv'), index=False)

## Saving `.sql`

In [265]:
con_to_sql(contributions,os.path.join(out_dir,'contributions.sql' ))
emo_to_sql(emotions_table,os.path.join(out_dir,'emotions.sql' ))
count_need_to_sql(country_needs,os.path.join(out_dir,'country_needs.sql' ))
per_need_to_sql(personal_needs,os.path.join(out_dir,'personal_needs.sql' ))

In [36]:
abreviation_dict = {
    'Media incompleta (incluyendo Media Técnica).': 'MedInc',
    'Básica completa': 'BasComp',
    'Básica completa.': 'BasComp',
    'Educación básica incompleta o inferior': 'BasInc',
    'Universitaria incompleta. Técnica completa': 'UniInc/TecComp',
    'Post Grado (Master, Doctor o equivalente)': 'Postg',
    '28': '',
    'Media completa. Técnica incompleta': 'MedComp/TecInc',
    'Media completa. Técnica incompleta.': 'MedComp/TecInc',
    'Universitaria completa.': 'UniComp',
    'Universitaria completa': 'UniComp',
    'Universitaria incompleta. Técnica completa.': 'UniInc/TecComp',
    'Media incompleta (incluyendo Media Técnica)': 'MedInc',
    'Educación básica incompleta o inferior.': 'BasInc',
    'Técnica completa': 'TecComp'
}

individuals_table['level'] = individuals_table['level'].replace(abreviation_dict)

In [37]:
individuals_table.to_csv(os.path.join(out_dir, 'individuals.csv'), index=False)